# Emojitranslate project

## Part one - learning to predict emojis from tweets

The following code adapted from the Osinga deep learning cookbook - using the Twitter API to sample EN language tweets that contain exactly one emoji

In [114]:
import random
import twitter
import emoji
# import itertools
import pandas as pd
from itertools import chain
from keras.preprocessing.sequence import pad_sequences
import numpy as np
#from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.utils import to_categorical
import keras.callbacks
#import json

import os
# import nb_utils
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, Embedding, GlobalMaxPooling1D#, Merge 
from keras.models import Model
from keras.layers.merge import Concatenate, Average

# from gensim.models import Word2Vec

In [30]:
all_tweets = pd.read_csv('data/emojis_homemade.csv')
all_tweets['emoji'].value_counts()

😂        35746
😭        14450
❤        11139
emoji     8154
😍         7890
🔥         7636
🤣         4974
🤔         3923
🙏         3607
😩         3584
💕         3578
😊         3392
👀         3127
💜         2936
✨         2924
👏         2887
🙄         2825
🖤         2638
💀         2628
😏         2554
🎉         2374
🙌         2310
😘         2265
💯         2147
👍         2026
💙         2013
😉         1925
💖         1878
🚨         1873
👇         1860
         ...  
◻            1
🗃            1
🀄            1
🚛            1
🛤            1
⏸            1
🈴            1
🚸            1
📏            1
⛎            1
🎍            1
🕖            1
🈲            1
🏨            1
➗            1
◀            1
🉐            1
🧙            1
🗂            1
💹            1
🚃            1
🚡            1
🖲            1
🛋            1
👷            1
🏬            1
🗜            1
🚥            1
🥠            1
🧚            1
Name: emoji, Length: 1054, dtype: int64

In [31]:
all_tweets.shape

(281488, 3)

In [32]:
all_tweets.head

<bound method NDFrame.head of         Unnamed: 0                                               text emoji
0              0.0  RT @mydear_bangtan: [VID] 181023 - Foi adicion...     ©
1              1.0  RT @WuYiFan_KrisBar: 181023 Kris Wu Studio upd...     💫
2              2.0  RT @TrendsPrabhas: Now you are watching Indian...     😎
3              3.0                                    dats for keeps      💛
4              4.0  RT @xruiztru: WHO ARMS THE SAUDIS?\n\n  🇺🇸US 5...     🏳
5              5.0  RT @xxxfreaknasty2: interracial couple go at i...     🌝
6              6.0               @Eric_Deshaun Holy shit no I think.      😩
7              7.0     RT @fanmutuals: army, follow who retweet this      👑
8              8.0  RT @_Simplykpop: Simply K-Pop harddrive dump\n...     🤩
9              9.0  Happy birthday nellie @thelittlegend, hope you...     ❤
10            10.0  RT @montparnasty: "I have passed through fire"...     💕
11            11.0  RT @Jason_Mckeown: Look how empty Vall

In [33]:
tweets = all_tweets.groupby('emoji').filter(lambda c:len(c) > 1000)
tweets['emoji'].value_counts()

😂        35746
😭        14450
❤        11139
emoji     8154
😍         7890
🔥         7636
🤣         4974
🤔         3923
🙏         3607
😩         3584
💕         3578
😊         3392
👀         3127
💜         2936
✨         2924
👏         2887
🙄         2825
🖤         2638
💀         2628
😏         2554
🎉         2374
🙌         2310
😘         2265
💯         2147
👍         2026
💙         2013
😉         1925
💖         1878
🚨         1873
👇         1860
         ...  
😳         1797
😅         1670
💪         1603
😁         1600
😱         1571
‼         1505
💛         1498
👌         1489
🗣         1454
🙃         1393
🤪         1353
😢         1304
☺         1287
😤         1278
✌         1240
🤗         1213
🤧         1160
😒         1138
💥         1131
😈         1121
©         1119
💔         1114
😫         1108
😌         1082
☹         1079
💗         1069
👉         1066
🙂         1064
🎶         1044
🤩         1007
Name: emoji, Length: 64, dtype: int64

In [34]:
max(tweets['text'], key=lambda t:len(t))

'RT @DingDongLive: LMAO  @TexasEDMFamily\n@FreakyDeakyFam\nGrab my hard/hybrid trap mix&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt; https://t.co/34rcxAVbgk https://t.co/ArQeh23L…'

In [35]:
chars = list(sorted(set(chain(*tweets['text']))))
char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
max_sequence_len = max(len(x) for x in tweets['text'])

emojis = list(sorted(set(tweets['emoji'])))
emoji_to_idx = {em: idx for idx, em in enumerate(emojis)}
emojis[:10]

#train_tweets, test_tweets = train_test_split(tweets, test_size=0.1)

['emoji', '©', '‼', '☹', '☺', '♥', '✌', '✨', '❤', '🎃']

Currently we've collected quite a lot of tweet data, so to speed up the prototyping of the model we're going to create a train/dev/test datasets of 10,000 tweets each.

In [36]:
train_tweets = tweets[0:10000]
dev_tweets = tweets[10000:20000]
test_tweets = tweets[20000:30000]

In [37]:
def data_generator(tweets, batch_size):
    while True:
        if batch_size is None:
            batch = tweets
            batch_size = batch.shape[0]
        else:
            batch = tweets.sample(batch_size)
        X = np.zeros((batch_size, max_sequence_len, len(chars)))
        y = np.zeros((batch_size,))
        for row_idx, (_, row) in enumerate(batch.iterrows()):
            y[row_idx] = emoji_to_idx[row['emoji']]
            for ch_idx, ch in enumerate(row['text']):
                X[row_idx, ch_idx, char_to_idx[ch]] = 1
        yield X, y

next(data_generator(tweets, 10))

(array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        ...,
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],


In [38]:
def create_char_cnn_model(num_chars, max_sequence_len, num_labels):
    char_input = Input(shape=(max_sequence_len, num_chars), name='char_cnn_input')
    
    conv_1x = Conv1D(128, 6, activation='relu', padding='valid')(char_input)
    max_pool_1x = MaxPooling1D(4)(conv_1x)
    conv_2x = Conv1D(256, 6, activation='relu', padding='valid')(max_pool_1x)
    max_pool_2x = MaxPooling1D(4)(conv_2x)

    flatten = Flatten()(max_pool_2x)
    dense = Dense(128, activation='relu')(flatten)
    preds = Dense(num_labels, activation='softmax', name='char_cnn_predictions')(dense)

    model = Model(char_input, preds)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc'])
    return model

char_cnn_model = create_char_cnn_model(len(char_to_idx), max_sequence_len, len(emojis))
char_cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
char_cnn_input (InputLayer)  (None, 163, 2222)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 158, 128)          1706624   
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 34, 256)           196864    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 8, 256)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               262272    
__________

In [46]:
early = keras.callbacks.EarlyStopping(monitor='loss',
                              min_delta=0.03,
                              patience=2,
                              verbose=1, mode='auto')

checkpoint = keras.callbacks.ModelCheckpoint(filepath='emoji_cnn.h5',
                                             monitor='val_acc',
                                             save_best_only = True)

#tensorboard = keras.callbacks.TensorBoard(log_dir='tensorboard_log',
#                                         histogram_freq=1,
#                                         embeddings_freq=1)

BATCH_SIZE = 512
char_cnn_model.fit_generator(
    data_generator(train_tweets, batch_size=BATCH_SIZE),
    validation_data = data_generator(dev_tweets, batch_size=BATCH_SIZE),
    validation_steps=int(dev_tweets.shape[0]/BATCH_SIZE),
    epochs=5,
    steps_per_epoch=len(train_tweets) / BATCH_SIZE,
    verbose=1, # was: verbose=2
    callbacks=[early, checkpoint]
)

Epoch 1/5
20/19 [==============================] - 328s 16s/step - loss: 3.0136 - acc: 0.3029 - val_loss: 3.1220 - val_acc: 0.3046


ValueError: If printing histograms, validation_data must be provided, and cannot be a generator.

In [45]:
pd.options.display.max_colwidth = 128
inspect_tweets = dev_tweets.sample(100)
predicted = char_cnn_model.predict_generator(data_generator(inspect_tweets, batch_size=None), steps=1)
show = pd.DataFrame({
    'text': inspect_tweets['text'],
    'true': inspect_tweets['emoji'],
    'pred': [emojis[np.argmax(x)] for x in predicted],
})
show = show[['text', 'true', 'pred']]
show.head(10)

,text,true,pred
23120,RT @PGATOUR: This is as unlucky as golf gets. \n\n#TOURVault https://t.co/1lyERpKge8,🙄,❤
23904,The worst champions league atmosphere I’ve ever heard at old Trafford #manu #overated,🙄,😂
24903,i think i go to school to get sick not smart,😂,😂
21950,"Someone said the Cake Island Arc in One Piece has made them hate cake...\nFor me, it makes me want to stuff my face with swe...",😂,😂
18236,RT @tribranchvo: she did NOT just do the “okurrrrr” ASMR style no she did not https://t.co/t1FtqLXUkD,💀,😂
16296,RT @wisconsinGodsPl: THE 1980 CUBAN BOAT LIFT WAS A IMMIGRATION CRISIS\n\nNOTICE THE HEADLINES IN THE PAPER\n\nPRESIDENT TRU...,🤔,❤
22613,@Humor_Silly Actually BJPWon Main Inka Haath Zyada Hai,🤣,😂
17512,text,emoji,emoji
21778,"RT @BillHai61993017: @RealSaavedra Imagine that, a DYING party trying to register DEAD people!!!!",😅,❤
19871,RT @Soumz: Today is Marshmallow appreciation day. Because every day is Marshmallow appreciation day \nأعطو حبكم للمارشملو h...,💙,❤


# Simple neural network - single Dense layer

Benchmark performance with the simplest neural network we can get

Let's divide up the train/dev/test sets so we're not relying on a generator

In [118]:
x_train = train_tweets["text"]
y_train = np.asarray(train_tweets["emoji"])
x_dev = dev_tweets["text"]
y_dev = np.asarray(dev_tweets["emoji"])
x_test = test_tweets["text"]
y_test = np.asarray(test_tweets["emoji"])

all_emojis = np.concatenate((y_train, y_dev, y_test), axis=0)

emoji_to_idx = {em: idx for idx, em in enumerate(emojis)}
#emojis[:10]

all_emojis_idx = np.zeros(all_emojis.shape[0])

for i in range (all_emojis.shape[0]):
    all_emojis_idx[i] = emoji_to_idx[all_emojis[i]]    

all_emojis_one_hot = to_categorical (all_emojis_idx)
    
y_train_idx = all_emojis_one_hot[0:10000,:]
y_dev_idx = all_emojis_one_hot[10000:20000,:]
y_test_idx = all_emojis_one_hot[20000:30000,:]

In [119]:
y_train_idx[0:10]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

Now let's start by one-hot encoding the text samples

In [120]:
tokenizer = Tokenizer(num_words = 1000)
tokenizer.fit_on_texts (x_train)

#x_train_sequences = tokenizer.texts_to_sequences(x_train)
#x_dev_sequences = tokenizer.texts_to_sequences(x_dev)
#x_test_sequences = tokenizer.texts_to_sequences(x_test)

x_train_one_hot = tokenizer.texts_to_matrix(x_train, mode='binary')
x_dev_one_hot = tokenizer.texts_to_matrix(x_dev, mode='binary')
x_test_one_hot = tokenizer.texts_to_matrix(x_test, mode='binary')




# remember to pad sequences

In [121]:
x_train_one_hot.shape

(10000, 1000)

In [122]:
y_train_idx.shape

(10000, 64)

In [126]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(1000,)))
model.add(Dense(64, activation = 'softmax'))
model.compile(optimizer = 'rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit (x_train_one_hot, y_train_idx,
                    epochs = 10,
                    batch_size = 512,
                    validation_data = (x_dev_one_hot, y_dev_idx))

Train on 10000 samples, validate on 10000 samples
Epoch 1/10
10000/10000 [==============================] - 1s 86us/step - loss: 3.9899 - acc: 0.1738 - val_loss: 3.8386 - val_acc: 0.2175
Epoch 2/10
10000/10000 [==============================] - 0s 36us/step - loss: 3.6715 - acc: 0.2107 - val_loss: 3.6105 - val_acc: 0.2208
Epoch 3/10
10000/10000 [==============================] - 0s 42us/step - loss: 3.4729 - acc: 0.2258 - val_loss: 3.5016 - val_acc: 0.2755
Epoch 4/10
10000/10000 [==============================] - 0s 39us/step - loss: 3.3450 - acc: 0.2944 - val_loss: 3.4162 - val_acc: 0.2989
Epoch 5/10
10000/10000 [==============================] - 0s 39us/step - loss: 3.2287 - acc: 0.3194 - val_loss: 3.3358 - val_acc: 0.3121
Epoch 6/10
10000/10000 [==============================] - 0s 41us/step - loss: 3.1207 - acc: 0.3363 - val_loss: 3.2617 - val_acc: 0.3283
Epoch 7/10
10000/10000 [==============================] - 0s 39us/step - loss: 3.0179 - acc: 0.3538 - val_loss: 3.1928 - val_acc

In [71]:
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit (input_train, y_train,
                     epochs = 10,
                     batch_size = 128,
                     validation_split = 0.2)

NameError: name 'max_features' is not defined